In [3]:
import pandas as pd

import pyteomics.mgf
from tqdm.notebook import tqdm

In [21]:
# infile = r"C:\git\msn_library\data\gnpslib\small_gnps.mgf"
infile = r"C:\git\msn_library\data\gnpslib\ALL_GNPS_NO_PROPOGATED.mgf"

In [44]:
ids, inchi, smiles, inchikey, names  = [], [], [], [], []

with pyteomics.mgf.MGF(infile) as f_in:
    for spectrum_dict in tqdm(f_in):
        if (
            int(spectrum_dict["params"]["libraryquality"]) <= 3 and
            float(spectrum_dict["params"]["pepmass"][0]) > 0 and
            len(spectrum_dict["m/z array"]) >= 3 and
            (
                spectrum_dict["params"]["inchi"] != "N/A" or
                spectrum_dict["params"]["smiles"] != "N/A" or
                spectrum_dict["params"]["inchiaux"] != "N/A"
            )
        ):
            ids.append(spectrum_dict["params"]["spectrumid"])
            inchi.append(spectrum_dict["params"]["inchi"])
            inchikey.append(spectrum_dict["params"]["inchiaux"])
            smiles.append(spectrum_dict["params"]["smiles"])
            names.append(spectrum_dict["params"]["name"])

df = pd.DataFrame({"gnps_libid": ids, "inchi": inchi, "inchi_key": inchikey, "Smiles": smiles, "compound_name": names, })

# remove adduct from name
df["compound_name"] = [str(name).rsplit(" M+")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 2M+")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 3M+")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 2M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 3M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" CollisionEnergy")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" [M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" [M+")[0] for name in df['compound_name']]

df = df.groupby(['inchi_key', 'inchi', 'Smiles', 'compound_name'])['gnps_libid'].apply(', '.join).reset_index()
df["entries"] = [str(lib).count(",") + 1 for lib in df["gnps_libid"]]

df.to_csv("data/gnpslib/gnps_library.csv", index=False)
df

0it [00:00, ?it/s]

,inchi_key,inchi,Smiles,compound_name,gnps_libid,entries
0,,,,(+)-vincadifformine,CCMSLIB00006710015,1
1,,,,(-)-Chaetominine,"CCMSLIB00010007511, CCMSLIB00010008035",2
2,,,,(-)-stepholidine,CCMSLIB00006710004,1
3,,,,"(1E,2E)-pentadec-2-enal oxime",CCMSLIB00004722163,1
4,,,,"(1R,1aR,2S,5R,5aR,6S,8aS,9R,10aR)-1,4-bis[(ace...",CCMSLIB00005463506,1
...,...,...,...,...,...,...
111864,ZSYPDXOLYNZQGO-CSWQYCJFSA-N,InChI=1S/C44H67N5O8S/c1-12-26(3)37-42(54)49-19...,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,Apratoxin B,"CCMSLIB00000001605, CCMSLIB00000001608",2
111865,ZWIADYZPOWUWEW-XVFCMESISA-N,"""InChI=1S/C9H15N3O11P2/c10-5-1-2-12(9(15)11-5)...",N/A,Cytidine 5_-diphosphate,CCMSLIB00000479613,1
111866,ZYGHJZDHTFUPRJ-UHFFFAOYSA-N,1S/C9H6O2/c10-9-6-5-7-3-1-2-4-8(7)11-9/h1-6H,C1=CC=C2C(=C1)C=CC(=O)O2,Coumarin,"CCMSLIB00010012785, CCMSLIB00010012786",2
111867,ZZYSLNWGKKDOML-UHFFFAOYSA-N,"""InChI=1S/C18H24ClN3O/c1-6-14-15(19)16(22(5)21...",CCc1c(c(n(n1)C)C(=O)NCc2ccc(cc2)C(C)(C)C)Cl,tebufenpyrad,CCMSLIB00006581742,1


In [45]:
df.shape

(111869, 6)

In [40]:
df = pd.read_csv("data/gnpslib/gnps_library.csv")